# LinkR - plugin Prescription

In [ ]:
# Librairies:

import pandas as pd
import seaborn as sns
import matplotlib as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots


# Drug Exposure

In [ ]:
drug_exposure = pd.read_csv('Data/drug_exposure.csv')
print(drug_exposure.shape) #(18229, 23)
'''
['drug_exposure_id', 'person_id', 'drug_concept_id',
       'drug_exposure_start_date', 'drug_exposure_start_datetime',
       'drug_exposure_end_date', 'drug_exposure_end_datetime',
       'verbatim_end_date', 'drug_type_concept_id', 'stop_reason', 'refills',
       'quantity', 'days_supply', 'sig', 'route_concept_id', 'lot_number',
       'provider_id', 'visit_occurrence_id', 'visit_detail_id',
       'drug_source_value', 'drug_source_concept_id', 'route_source_value',
       'dose_unit_source_value']
'''
drug_exposure.person_id.value_counts()
'''
person_id
 7155255168997124770    1182
 4783904755296699562     791
 8090044958540695372     716
-3908355835367628651     687
 7131048714591189903     644
                        ... 
-2575767131279873665      37
-4183220989401122518      34
 1740609625029317924      26
 3912882389848878631      24
-8769042030325953499      18
'''
drug_exposure.info()
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18229 entries, 0 to 18228
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   drug_exposure_id              18229 non-null  int64  
 1   person_id                     18229 non-null  int64  
 2   drug_concept_id               18229 non-null  int64  
 3   drug_exposure_start_date      18229 non-null  object 
 4   drug_exposure_start_datetime  18229 non-null  object 
 5   drug_exposure_end_date        18229 non-null  object 
 6   drug_exposure_end_datetime    18229 non-null  object 
 7   verbatim_end_date             0 non-null      float64
 8   drug_type_concept_id          18229 non-null  int64  
 9   stop_reason                   0 non-null      float64
 10  refills                       0 non-null      float64
 11  quantity                      18220 non-null  float64
 12  days_supply                   0 non-null      float64
 13  sig                           0 non-null      float64
 14  route_concept_id              18229 non-null  int64  
 15  lot_number                    0 non-null      float64
 16  provider_id                   0 non-null      float64
 17  visit_occurrence_id           18229 non-null  int64  
 18  visit_detail_id               0 non-null      float64
 19  drug_source_value             18229 non-null  object 
 20  drug_source_concept_id        18229 non-null  int64  
 21  route_source_value            18223 non-null  object 
 22  dose_unit_source_value        18220 non-null  object 
 '''

drug_exposure.head()

liste_non_null = ['drug_exposure_id', 'person_id', 'drug_concept_id',  'drug_exposure_start_date', 'drug_exposure_start_datetime',
       'drug_exposure_end_date', 'drug_exposure_end_datetime', 'drug_type_concept_id',   'quantity',
       'route_concept_id', 'visit_occurrence_id', 'drug_source_value', 'drug_source_concept_id', 'route_source_value', 'dose_unit_source_value']


In [ ]:
drug_exposure.describe()


### Patient 1: 3912882389848878631
On se concentre sur un premier cas avec 24 lignes pour 1 personne: 
Attention, c'est un integer, pas un texte. Ne pas mettre de ''


In [ ]:
patient1 = drug_exposure.loc[drug_exposure['person_id'] == 3912882389848878631, liste_non_null]
'''
patient1.columns
Index(['drug_exposure_id', 'person_id', 'drug_concept_id',
       'drug_exposure_start_date', 'drug_exposure_start_datetime',
       'drug_exposure_end_date', 'drug_exposure_end_datetime',
       'drug_type_concept_id', 'quantity', 'route_concept_id',
       'visit_occurrence_id', 'drug_source_value', 'drug_source_concept_id',
       'route_source_value', 'dose_unit_source_value'],
      dtype='object')
'''
patient1

In [ ]:
patient1[patient1.dose_unit_source_value == 'TAB']
# 9 lignes, des exposure_id différentes, avec des dates différents (2129 ????)
# drug_concept_id being the name of the drug I guess, Quantity in Tablettes.


In [ ]:
patient1_drug1 = patient1.loc[patient1.drug_concept_id==40232756,['drug_exposure_start_datetime', 'drug_exposure_end_datetime', 'quantity']].sort_values(by='drug_exposure_start_datetime')



patient1_drug1['cumulative'] = [0.5, 1.5, 3.0, 4.0]



# ne fonctionne pas comme ça, car les date ne sont pas correct.... il faut prendre datetime déjà, puis trier en fonction du temps.
# il faut faire avec plotly un graph pour choisir le medoc.
# il faut calculer le cumulatif ou non ?
patient1_drug1


In [ ]:

fig = make_subplots(rows = 1, cols = 2, subplot_titles=('not cumulative','cumulative'))
fig.add_trace(go.Scatter(x = patient1_drug1.drug_exposure_start_datetime, y = patient1_drug1.quantity), row=1, col=1)
fig.add_trace(go.Scatter(x = patient1_drug1.drug_exposure_start_datetime, y = patient1_drug1.cumulative), row=1, col=2)

fig.show()

In [ ]:
patient1 = patient1.sort_values(by='drug_exposure_start_datetime')
number_types = patient1.dose_unit_source_value.describe()['unique']

fig3 = make_subplots(rows = 1, cols = number_types, subplot_titles=(patient1.dose_unit_source_value.value_counts().index))
colonne = 1

for drug_type in patient1.dose_unit_source_value.value_counts().index:
    #print(drug_type)

    for d in patient1[patient1.dose_unit_source_value == drug_type].drug_concept_id.unique():
        temp = patient1[patient1.drug_concept_id == d]
        #print(temp.dose_unit_source_value.unique())
        fig3.add_trace(go.Scatter(x = temp.drug_exposure_start_datetime, y = temp.quantity, name=f'{d}, {temp.dose_unit_source_value.unique()}', legendgroup=drug_type), row=1, col=colonne)

    colonne += 1

fig3.show()

# Add curseur de temps, add legendgroup, add selection de chaque drug

### Drug_exposure start and end can be different

In [ ]:
# drug_exposure.drug_exposure_start_datetime.value_counts()
#drug_exposure.drug_exposure_end_datetime.value_counts()
print(drug_exposure.drug_exposure_start_datetime[0], drug_exposure.drug_exposure_end_datetime[0])
# Can be different....

# Dose Era

A Dose Era is defined as a span of time when the Person is assumed to be exposed to a constant dose of a specific active ingredient.

Dose Form information is not taken into account. So, if the patient changes between different formulations, or different manufacturers with the same formulation, the Dose Era is still spanning the entire time of exposure to the Ingredient.

In [ ]:
dose_era = pd.read_csv('Data/dose_era.csv')
dose_era.head()

dose_era.shape #(117, 7)

In [ ]:
dose_era.unit_concept_id.value_counts()
# 8576 are all in mg.... This is good :) I can go back to drug_concept id, dose_value....

In [ ]:
dose_era.drug_concept_id.value_counts()

In [ ]:
dose_era[dose_era.person_id == 3912882389848878631] # my patient is not recognized, neither my drug_cncept_id...why ????
#patient1[patient1.dose_unit_source_value == drug_type]
dose_era[dose_era.drug_concept_id == 19127213]
# ne pas mettre de guillements, ça ne fonctionne pas...

# Drug era

**active ingredient**, successive period of drug_exposure to the same ingredent gives drug era

A Drug Era is defined as a span of time when the Person is assumed to be exposed to a particular active ingredient. A Drug Era is not the same as a Drug Exposure:
Exposures are individual records corresponding to the source when Drug was delivered to the Person, while successive periods of Drug Exposures are combined under certain rules to produce continuous Drug Eras.

-> not the same concept_id because here we talk about an ingredient, not a particular drug... diazepam vs diazepam tablet 10mg.
Example:
drug_era: person 3912882389848878631 got 2 exposure to diazepam 723013 between 2129-01-03	2129-01-05
[6598	-7540355434127132424	3912882389848878631	723013	2129-01-03	2129-01-05	2	0]
drug_exposure


In [ ]:
drug_era = pd.read_csv('Data/drug_era.csv')

drug_era.shape #(7931, 7)
drug_era.head()

In [ ]:
drug_era.drug_concept_id.value_counts()

In [ ]:
drug_era[drug_era.drug_concept_id == 19127213] # cannot be found...

drug_era[drug_era.person_id == 3912882389848878631] # -> ok...

In [ ]:
temp_1 = drug_era.loc[drug_era.person_id == 3912882389848878631, 'drug_concept_id'].unique() #16
temp_1.sort()
'''
drug_era
(array([  723013 - 	diazepam as an ingredient,   734275,   923645,   967823,  1124957,  1125315,
         1177480,  1367571,  1560524, 19111620, 19137312, 36878782,
        40164828, 40166605, 42873956, 43531944]),

drug_exposure
 array([  723020 - diazepam 10 MG Oral Tablet RxNorm,   734281,  1127433, 19019050, 19019418, 19077513,
        19079250, 19126251, 19127213, 19137328, 21125875, 36249736,
        40221329, 40232756, 43011850, 45775137]))
'''
temp_2 = drug_exposure.loc[drug_exposure.person_id == 3912882389848878631, 'drug_concept_id'].unique() #16
temp_2.sort()

# all the same

temp_1, temp_2


In [ ]:
temp_3 = drug_exposure.loc[drug_exposure.person_id == 3912882389848878631]
temp_3
#temp_3.loc[temp_3.drug_concept_id == 723020]

In [ ]:
temp_exp = drug_exposure.person_id.unique()
len(temp_exp) #100
temp_drug = drug_era.person_id.unique()
len(temp_drug) #100
temp_dose = dose_era.person_id.unique()
len(temp_dose) #44